In [239]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.integrate import simps

# Helper functions


def ensemble_average(all_data, validity_fn = lambda x : x['MaxJump']<0.4) :
    """
        Take a PAFI csv data dump and produce ensemble average and standard deviation
        `data` : pandas DataFrame in PAFI format
        `validity_criteria` : a function acting on a Dataframe which returns valid data
    """
    
    nWorkers = max(set(all_data['WorkerID'])) + 1
    
    if not validity_fn is None:
        data = all_data[validity_fn(all_data)]
    else:
        data = all_data.copy()
    
    
    # Every field before nWorkers is a parameter
    nParams = np.where(data.columns=='WorkerID')[0][0] 
    
    # Only use WorkerID==0 values as np.unique is slow
    pValues = np.unique(data[data['WorkerID']==0].to_numpy()[:,:nParams],axis=0)
    
    
    param_names = list(data.columns[:nParams])
    
    field_names = list(data.columns[nParams+1:])
    
    ens_columns = param_names + field_names + [f+"_std" for f in field_names]
    
    ens_data = []
    
    raw_data = data.drop('WorkerID', 1).to_numpy()
    
    for p in pValues:
        sel = (raw_data[:,:nParams] == p).min(axis=1)
        ens_data += [list(raw_data[sel].mean(axis=0)) + list(raw_data[sel].std(axis=0)[nParams:])]
    
    ensemble_average = pd.DataFrame(data = np.r_[ens_data], columns = ens_columns)
    
    ensemble_average.attrs = {'nParams' : nParams, 'nWorkers' : nWorkers, 'Parameters' : param_names}
    
    return ensemble_average
    


def integrate(ens ):
    """
        Integrate ensemble average free energy gradient with respect to r, with error
        `ens` : ensemble average dataset
    """
    if (not 'aveF' in ens.columns) or (not 'ReactionCoordinate' in ens.columns):
        print("Need free energy gradient (aveF) and ReactionCoordinate in PAFI data!")
        return
    aveF = ens['aveF'].to_numpy()
    r = ens['ReactionCoordinate'].to_numpy()
    
    
    
    

    

# Load in the raw PAFI `.csv` output and display the first few rows

In [242]:
data = pd.read_csv('dumps/raw_data_output_0.csv')
data.head()

,Lambda,ReactionCoordinate,Temperature,WorkerID,MaxDev,MaxJump,MinEnergy,Valid,aveF,avePsi,dXTangent,f_dV_0,postT,preT,stdF
0,0.0,0.0,1,0,0.015636,0.017823,-5656.91,1,0.087837,1.0,3.975620e-15,-0.188112,1.06143,0.961009,0.000017
1,0.0,0.0,1,1,0.018068,0.018070,-5656.91,1,0.111887,1.0,4.536800e-15,-0.192350,1.06852,0.924825,0.000089
2,0.2,0.0,1,0,0.018771,0.018052,-5656.91,1,0.119210,1.0,7.045490e-15,-0.193341,1.04348,0.936385,0.000070
3,0.2,0.0,1,1,0.016608,0.018142,-5656.91,1,0.123983,1.0,2.885640e-15,-0.191196,1.05282,0.945967,0.000128
4,0.4,0.0,1,0,0.016999,0.018127,-5656.91,1,0.116088,1.0,4.514530e-15,-0.188713,1.04157,0.936095,0.000184


# Integrate ensemble average data

In [243]:
average_data = ensemble_average(data)
average_data.head()

,Lambda,ReactionCoordinate,Temperature,MaxDev,MaxJump,MinEnergy,Valid,aveF,avePsi,dXTangent,...,MaxJump_std,MinEnergy_std,Valid_std,aveF_std,avePsi_std,dXTangent_std,f_dV_0_std,postT_std,preT_std,stdF_std
0,0.0,0.0,1.0,0.016852,0.017947,-5656.91,1.0,0.099862,1.000000,4.256210e-15,...,0.000124,0.0,0.0,0.012025,0.000000,2.805900e-16,0.002119,0.003545,0.018092,0.000036
1,0.0,0.5,1.0,0.035097,0.040746,-5656.32,1.0,-0.004150,0.977995,1.359590e-15,...,0.000006,0.0,0.0,0.000989,0.001420,2.781600e-16,0.000823,0.004040,0.014818,0.000002
2,0.2,0.0,1.0,0.017689,0.018097,-5656.91,1.0,0.121596,1.000000,4.965565e-15,...,0.000045,0.0,0.0,0.002386,0.000000,2.079925e-15,0.001073,0.004670,0.004791,0.000029
3,0.2,0.5,1.0,0.037481,0.040975,-5656.32,1.0,-0.003651,0.976070,3.533065e-15,...,0.000036,0.0,0.0,0.004196,0.005084,1.745955e-15,0.002546,0.017310,0.016783,0.000016
4,0.4,0.0,1.0,0.016437,0.018039,-5656.91,1.0,0.113721,1.000000,7.972915e-15,...,0.000088,0.0,0.0,0.002368,0.000000,3.458385e-15,0.001040,0.003985,0.002219,0.000040


# Thermodynamic Integration

In [245]:
r = average_data['ReactionCoordinate']
lam = average_data[r==0.]['Lambda']
dV = average_data[r==0.]['f_dV_0']
tdV_g = simps(dV,lam)

lam = average_data[r==0.5]['Lambda']
dV = average_data[r==0.5]['f_dV_0']
tdV_s = simps(dV,lam)

print(tdV_s-tdV_g)



0.028876333333333337
